In [ ]:
0.# Install TensorFlow Datasets if not already installed
!pip install tensorflow-datasets

# Import required libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load and convert the CIFAR-10 dataset to NumPy arrays
# Load the entire dataset at once by setting batch_size=-1
dataset, info = tfds.load('cifar10', with_info=True, batch_size=-1, as_supervised=True)

# Convert the dataset to NumPy arrays
(img_train, label_train), (img_test, label_test) = tfds.as_numpy((dataset['train'], dataset['test']))

# Display dataset information
print(info)

# Check the shapes of the arrays to verify the data
print(f"Training images shape: {img_train.shape}")
print(f"Training labels shape: {label_train.shape}")
print(f"Test images shape: {img_test.shape}")
print(f"Test labels shape: {label_test.shape}")

# Example: Display the first image in the training dataset
plt.figure(figsize=(2, 2))
plt.imshow(img_train[100])
plt.title(f"Label: {label_train[100]}")
plt.show()

# 1. Data Preprocessing

from tensorflow.keras.utils import to_categorical

# Create one-hot encoded labels
label_train_onehot = to_categorical(label_train, num_classes=10)
label_test_onehot = to_categorical(label_test, num_classes=10)

# Check the shape of the new labels
print(f"One-hot encoded training labels shape: {label_train_onehot.shape}")
print(f"One-hot encoded test labels shape: {label_test_onehot.shape}")


# Normalize the image data
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Verify the normalization
print(f"Training images shape: {img_train.shape}")
print(f"Training images min value: {img_train.min()}")
print(f"Training images max value: {img_train.max()}")
print(f"Test images shape: {img_test.shape}")
print(f"Test images min value: {img_test.min()}")
print(f"Test images max value: {img_test.max()}")

# Define the data augmentation model
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

# Apply data augmentation to some images and visualize the results
fig, axes = plt.subplots(3, 5, figsize=(8, 8))
for i in range(3):
    orig_image = img_train[i]
    axes[i, 0].imshow(orig_image)
    axes[i, 0].set_title('Original')
    axes[i, 0].axis('off')
    for j in range(1, 5):
        augmented_image = data_augmentation(tf.expand_dims(orig_image, 0))
        axes[i, j].imshow(augmented_image[0])
        axes[i, j].set_title(f'Augmented {j}')
        axes[i, j].axis('off')
plt.tight_layout()
plt.show()

# Create visualisations of some images and labels

fig, axes = plt.subplots(10, 10, figsize=(10, 10))
classes = np.unique(label_train)
for i, cls in enumerate(classes):
    idxs = np.where(label_train == cls)[0]
    random_idxs = np.random.choice(idxs, 10, replace=False)
    for j, idx in enumerate(random_idxs):
        axes[i, j].imshow(img_train[idx])
        axes[i, j].axis('off')
    fig.text(0.08, 0.85 - (i * 0.085), f"Class {cls}", va='center', ha='left', fontsize=8)
plt.show()

# 2. Model Architecture

from tensorflow.keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# input_shape = (32, 32, 3)

model = Sequential([

Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
BatchNormalization(),
Conv2D(128, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
MaxPooling2D((2, 2)),
Dropout(0.2),

# Second block
Conv2D(128, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
Conv2D(128, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
MaxPooling2D((2, 2)),
Dropout(0.3),

# Third block
Conv2D(256, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
Conv2D(256, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
MaxPooling2D((2, 2)),
Dropout(0.4),

# Fourth block
Conv2D(512, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
Conv2D(512, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
MaxPooling2D((2, 2)),
Dropout(0.5),

# Fifth block
Conv2D(512, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
Conv2D(512, (3, 3), activation='relu', padding='same'),
BatchNormalization(),
MaxPooling2D((2, 2)),
Dropout(0.5),

(Flatten()),

Dense(128, activation='relu'),
BatchNormalization(),
Dropout(0.5),
Dense(10, activation='softmax')
])

model.summary()

# 3. Model Training

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Create a custom data generator
def augment_data(images, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                          num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.prefetch(tf.data.AUTOTUNE)

# Split the data into training and validation sets
validation_split = 0.1
num_val = int(len(img_train) * validation_split)
train_images, val_images = img_train[:-num_val], img_train[-num_val:]
train_labels, val_labels = label_train_onehot[:-num_val], label_train_onehot[-num_val:]

# Create train dataset with augmentation
train_dataset = augment_data(train_images, train_labels, batch_size=64)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(64)


# Add the model.fit() call here
history = model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train (Model 1)', 'Validation (Model 1)'], loc='upper left')

# 4. Model Evaluation

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Get the true labels
gt = np.argmax(label_test_onehot, axis=1)

# Predict the output for the test split
predictions = model.predict(img_test)
predictions = np.argmax(predictions, axis=1)

# Calculate and print the metrics
accuracy = accuracy_score(gt, predictions)
precision = precision_score(gt, predictions, average='weighted')
recall = recall_score(gt, predictions, average='weighted')
f1 = f1_score(gt, predictions, average='weighted')


print(f"Model accuracy: {accuracy:.4f}")
print(f"Model precision: {precision:.4f}")
print(f"Model recall: {recall:.4f}")
print(f"Model F1 score: {f1:.4f}")



# Predict the output for the test split
predictions = model.predict(img_test)

print(f"Predictions shape: {predictions.shape}")

# Convert predictions and true labels to class indices
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(label_test_onehot, axis=1)

# Calculate accuracy score
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Test accuracy: {accuracy:.4f}")

# Create a confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()



#from keras.backend import clear_session
#clear_session()